In [1]:
from Modules.Heating import Heat_Pump
from Modules.Heating import Controller
import numpy as np
import matplotlib.pyplot as plt

max_heat_pump_power = 305 # in kW

Heat_Pump = Heat_Pump("Data/Heat_Pump/EcoAir622M/COP.csv")

Controller = Controller(Heat_Pump, max_heat_pump_power)

time, heating, electricity, COP = Controller.controller()

print('Total electricity usage ' + str(np.sum(electricity)) + ' kWh/yr')

effective_COP = np.sum(heating) / np.sum(electricity)
print('Effective COP: ' + str(effective_COP))

plt.figure(0)
plt.plot(time, COP)
plt.figure(1)
plt.plot(time, heating)
plt.figure(2)
plt.plot(time, electricity)
plt.figure(3)
plt.plot(time, np.sort(heating))

AttributeError: 'Controller' object has no attribute 'max_heat_pump_power'